In [ ]:
#resnet_transf.py 
#Faz transfer learning usando ResNet50.
#https://medium.com/abraia/first-steps-with-transfer-learning-for-custom-image-classification-with-keras-b941601fcad5
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import numpy as np;
import keras as keras; import keras.backend as K;
from keras import optimizers, callbacks, regularizers;
from keras.regularizers import l2;
from keras.models import Sequential, Model;
from keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten;
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image

In [ ]:
def leCsv(nomeDir, nomeArq, nl=0, nc=0):
    st=os.path.join(nomeDir,nomeArq);
    arq=open(st,"rt"); lines=arq.readlines(); arq.close(); n=len(lines)
    linhas_separadas=[]
    for linha in lines:
       linha=linha.strip('\n'); linha=linha.split(';'); linhas_separadas.append(linha);
    ay=np.empty((n),dtype='float32'); ax=np.empty((n,nl,nc,3),dtype='float32');
    for i in range(len(linhas_separadas)):
       linha=linhas_separadas[i];
       img_path=os.path.join(nomeDir,linha[0])
       t = image.load_img(img_path, target_size=(nl,nc))
       x = image.img_to_array(t)
       x = np.expand_dims(x, axis=0)
       ax[i] = preprocess_input(x)
       ay[i] = np.float32(linha[1]); #0=m ou 1=f
    return ax, ay
